# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


### Цели исследования
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

### Описание данных


| Название столбца       | Значение                                   |
|------------------------|--------------------------------------------|
| ***children***         | количество детей в семье                   |
| ***days_employed***    | общий трудовой стаж в днях                 |
| ***dob_years***        | возраст клиента в годах                    |
| ***education***        | уровень образования клиента                |
| ***education_id***     | идентификатор уровня образования           |
| ***family_status***    | семейное положение                         |
| ***family_status_id*** | идентификатор семейного положения          |
| ***gender***           | пол клиента                                |
| ***income_type***      | тип занятости                              |
| ***debt***             | имел ли задолженность по возврату кредитов |
| ***total_income***     | ежемесячный доход                          |

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [202]:
# импортируем нужные библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer('russian').stem  # создаем объект SnowballStemmer
from pymystem3 import Mystem
mystem = Mystem().lemmatize # создаем объект Mystem

In [203]:
# прочитаем предоставленный датасет
data = pd.read_csv('../data/data.csv')

# посмотрим информацию о содержании колонок
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Датасет содержит 21525 сэмплов и 12 столбцов, из них 2 столбца с данными типа float64, 5 с int64 и 5 с данными типа object.
В столбцах days_employed и total_income присутствуют пропуски данных, причем в обоих столбцах содержится одинаковое количество строк с пропусками - 2174.
Также, как минимум, в столбцах children, debt и dob_years выглядит избыточным использование int64.
Давайте посмотрим на содержание датасета и статистические данные численных столбцов.

In [204]:
data.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [205]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Из полученных данных можно сделать следующие выводы:**
1. У столбцов children, dob_years, education_id, family_status_id, debt максимальные значения никогда не превысят int64, так что можем смело конвертировать их в int8 сэкономив память.
2. В столбце children минимальное значение равно -1, что явно говорит о наличии недостоверных/ошибочных данных в данном столбце.
3. Столбец days_employed выглядит довольно странно, очень много отрицательных и слишком больших значений.
4. В столбце education необходимо привести данные к нижнему регистру для дальнейшего удобства работы с ними

Для начала преобразуем типы данных для экономии памяти.
Для наглядности создадим датафрейм, содержащий информацию об исходном датасете и датасете с преобразованными типами данных.

In [206]:
data_compressed = data.astype({'children': 'int8', 'days_employed': 'float32', 'dob_years': 'int8', 'education_id': 'int8', 'family_status_id': 'int8', 'debt': 'int8', 'total_income': 'float32'})
pd.DataFrame(data=[
    data.memory_usage(),
    data_compressed.memory_usage(),
    data.memory_usage() - data_compressed.memory_usage(),
    100 / (data.memory_usage() / (data.memory_usage() - data_compressed.memory_usage()))
    ]
).rename({0: 'before_optimization', 1: 'after_optimization', 2: 'compressed_bytes', 3: '%_compress'})

,Index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
before_optimization,128.0,172200.0,172200.0,172200.0,172200.0,172200.0,172200.0,172200.0,172200.0,172200.0,172200.0,172200.0,172200.0
after_optimization,128.0,21525.0,86100.0,21525.0,172200.0,21525.0,172200.0,21525.0,172200.0,172200.0,21525.0,86100.0,172200.0
compressed_bytes,0.0,150675.0,86100.0,150675.0,0.0,150675.0,0.0,150675.0,0.0,0.0,150675.0,86100.0,0.0
%_compress,0.0,87.5,50.0,87.5,0.0,87.5,0.0,87.5,0.0,0.0,87.5,50.0,0.0


Простым преобразованием типов данных удалось уменьшить потребление памяти от 50% до 87.5%.

In [207]:
data_compressed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int8   
 1   days_employed     19351 non-null  float32
 2   dob_years         21525 non-null  int8   
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int8   
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int8   
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int8   
 10  total_income      19351 non-null  float32
 11  purpose           21525 non-null  object 
dtypes: float32(2), int8(5), object(5)
memory usage: 1.1+ MB


Теперь датасет использует 1.1+ MB памяти вместо 2+ MB. Мы добились экономии памяти в ~45%.
Далее будем использовать "сжатый" датасет.

In [208]:
data = data_compressed

 Теперь приведем данные в столбце education к нижнему регистру для дальнейшего удобства работы с ними.

In [209]:
data['education'] = data['education'].apply(lambda x: x.lower())
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Ранее мы обнаружили пропуски в столбцах days_employed и total_income, давайте рассмотрим их поподробнее.
Учитывая что количество пропусков в столбцах одинаково, можно предположить что между ними есть зависимость и в строке,
где пропущено значение столбца days_employed, также пропущено значение столбца total_income.
Чтобы в этом убедиться выберем все строки, где пропущенны значения в обоих столбцах. Количество строк должно быть равно 2174.
Если будет больше, значит значения пропущены в разных строчках.
Давайте проверим это.


In [210]:

data.query('days_employed.isna() and total_income.isna()').children.count()


2174

Действительно между пропусками в столбцах days_employed и total_income есть прямая зависимость.
Теперь необходимо узнать, к какому типу пропусков относятся наши данные.
Для этого выберем все строки с пропущенными значениями в одном или двух столбцах и проверим есть ли совпадения к какому-то одному классу.
Для наглядности создадим для этого датафрейм.

In [211]:
# список колонок по которым будем провоидть анализ
columns = ['family_status', 'debt', 'income_type', 'gender', 'family_status', 'education', 'dob_years', 'children']
columns = ['family_status', 'debt', 'income_type', 'gender', 'education', 'dob_years']
# создаем отдельный датафрейм, состоящий из строк с пропущенными значениями в колонках
empty_data = data.query('days_employed.isna() and total_income.isna()')

# считаем количество пропущенных значений для каждого из значений анализируемой колонки
for column in columns:
    display(f"{'='*40} Column: {column} {'='*40}")
    display(
        pd.DataFrame(
            data=[
                data[column].value_counts() - empty_data[column].value_counts()
            ]
        ).rename({column: 'Количество пропущенных значений'}))

'======================================== Column: family_status ========================================'

,женат / замужем,гражданский брак,Не женат / не замужем,в разводе,вдовец / вдова
Количество пропущенных значений,11143,3735,2525,1083,865


'======================================== Column: debt ========================================'

,0,1
Количество пропущенных значений,17780,1571


'======================================== Column: income_type ========================================'

...

,F,M,XNA
Количество пропущенных значений,12752.0,6598.0,NaN


'======================================== Column: education ========================================'

,высшее,начальное,неоконченное высшее,среднее,ученая степень
Количество пропущенных значений,4716.0,261.0,675.0,13693.0,NaN


'======================================== Column: dob_years ========================================'

,0,19,20,21,22,23,24,25,26,27,...,66,67,68,69,70,71,72,73,74,75
Количество пропущенных значений,91.0,13.0,46.0,93.0,166.0,218.0,243.0,334.0,373.0,457.0,...,163.0,151.0,90.0,80.0,62.0,53.0,31.0,7.0,NaN,NaN


Из полученных данных мы можем сделать вывод, что имеем дело со случайной ошибкой т.к. есть зависимость
между пропусками в колонках days_employed и total_income, но нет зависимости от значений собственного столбца.

## Давайте заполним пропуски.
Для начала займемся пропусками столбца total_income.
Наиболее точно можно заполнить пропуски медианной зарплатой для датафрейма, сгруппированного по типу занятости, гендеру и образованию.
В случае, если ежемесячный доход не зависит от гендера, его можно будет исключить из группировки и посчитать медиану только по типу занятости и образованию.

In [212]:
data.groupby(['income_type', 'gender', 'education'])['total_income'].median()

income_type      gender  education          
безработный      F       высшее                 202722.515625
                 M       среднее                 59956.992188
в декрете        F       среднее                 53829.128906
госслужащий      F       высшее                 157841.437500
                         начальное               89618.968750
                         неоконченное высшее    136599.125000
                         среднее                125831.812500
                         ученая степень         111392.234375
                 M       высшее                 214706.640625
                         начальное              190966.656250
                         неоконченное высшее    177320.625000
                         среднее                169651.953125
компаньон        F       высшее                 189893.468750
                         начальное              134012.000000
                         неоконченное высшее    170471.625000
                         

Как можем убедиться во всех группах, кроме пенсионеров, ежемесячный доход порой значительно зависит от гендера.
Так что оставляем группировку по гендеру и заполняем пропущенные значения медианным значением.
Но у нас есть один мужчина предприниматель с высшим образованием, медиану ежемесечного дохода которого нельзя посчитать из-за нехватки данных.
Предположим что в предпринимательстве нет зависимости ежемесячного дохода от гендера, так что возьмем для него медианную ежемесячнаую зарплату
девушек предпринимателей с таким же образованием.

In [213]:
# присваиваем медианную зарплату вместо пропущенных значений
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type', 'gender', 'education'])['total_income'].transform('median'))
# присваиваем медианную зарплату оставшемуся предпринимателю
data.loc[data['total_income'].isna(), 'total_income'] = 499163.156250
# убедимся что пропущенных значений в столбце total_income больше нет
data.query('total_income.isna()').count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Теперь разберемся с данными в столбце days_employed.
В данном столбце содержится информация об общем трудовом стаже в днях.

In [214]:
data['days_employed'].describe()

count     19351.000000
mean      63046.500000
std      140827.312500
min      -18388.949219
25%       -2747.423584
50%       -1203.369507
75%        -291.095947
max      401755.406250
Name: days_employed, dtype: float64

С данными явно что-то не так. Большинство данных имеют отрицательные значения и явно выражены не в днях.
Предположим, что данные из этого столбца надо взять по модулю и выражены они не в днях, а в часах, т.о. нужно поделить значения на количество часов в сутках, т.е. 24 и округлить.

In [215]:
data['days_employed'] = data['days_employed'].abs() / 24
data['days_employed'].describe()

count    19351.000000
mean      2788.113770
std       5792.953125
min          1.005901
25%         38.625389
50%         91.425850
75%        230.745117
max      16739.808594
Name: days_employed, dtype: float64

Так-то лучше. В среднем трудовой стаж составляет почти 16 лет.
Посмотрим на пропущенные значения.

In [233]:
data.groupby(['income_type', 'gender'])['days_employed'].mean()

income_type      gender
безработный      F         16470.951172
                 M         14063.519531
в декрете        F           137.365005
госслужащий      F           146.502258
                 M           128.359589
компаньон        F            90.698952
                 M            83.400375
                 XNA          98.275024
пенсионер        F         15224.634766
                 M         15137.311523
предприниматель  F            21.702003
                 M                  NaN
сотрудник        F           105.136024
                 M            84.100830
студент          M            24.114647
Name: days_employed, dtype: float32

**Вывод**

## Шаг 2. Предобработка данных

### Обработка пропусков

**Вывод**

### Замена типа данных

**Вывод**

### Обработка дубликатов

**Вывод**

### Лемматизация

**Вывод**

### Категоризация данных

**Вывод**

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

**Вывод**

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Вывод**

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Вывод**

- Как разные цели кредита влияют на его возврат в срок?

**Вывод**

## Шаг 4. Общий вывод

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.